In [4]:
# !pip install imageio

In [3]:
# $ sudo apt update && sudo apt install -y \
#     texlive-latex-base \
#     texlive-fonts-recommended \
#     texlive-fonts-extra \
#     texlive-latex-extra \
#     dvipng \
#     cm-super

# Imports

In [65]:
from visualize_original_systems import (
    load_dataset,
    plot_1d_heatmap,
    plot_1d_heatmap_anim,
    plot_2d_heatmap_anim
)

# Define dataset paths
data_paths = {
    1: "data/1",
    2: "data/2",
    3: "data/3"
}

# Define colormaps for visualization
colormaps = {
    1: "jet",
    2: "jet",
    3: "jet"
}

In [ ]:
from derivative_approximators import (
    TikhonovDiff,
    FiniteDiff,
    ConvSmoother,
    PolyDiff,
    PolyDiffPoint
)

from build_theta_linear_system import (
    build_Theta,
    build_linear_system
)

from sparse_regression_solvers import (
    STRidge,
    TrainSTRidge
)

from utilities_1D import (
    print_pde,
    get_pde_string, 
    normalize_data
)

from visualize_1D import (
    plot_comparison_heatmaps,
    animate_comparison,
    plot_pde_terms, 
    plot_candidate_terms, 
    plot_residual_over_time,
    scatter_true_vs_pred
)

In [ ]:
from utilities_2D import (
    TrainSTRidge_multioutput,
    build_linear_system_2D, 
    downsample_3d
)

from visualize_2D import (
    load_dataset,
    animate_comparison_heatmaps_2D,
    plot_candidate_terms_PDE3,
    plot_candidate_terms_spatial,
    plot_residual_over_time_2D,
    animate_comparison_2D_slice,
    plot_pde_terms_multioutput,
    scatter_true_vs_pred_with_footnote
)


In [ ]:
import numpy as np
from numpy import linalg as LA
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import imageio
import imageio.v2 as imageio
import shutil

import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

# Enable LaTeX rendering
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
})

---

# Animations of Mystery (Original) Dynamical Systems

In [70]:
# Main execution
results_dir = "results/new_visualizations"
os.makedirs(results_dir, exist_ok=True)

# Processing System 1
print("Processing system 1...")
U1, V1, X1, Y1, T1 = load_dataset(data_paths[1])
plot_1d_heatmap(U1, T1, X1, "system_1_U", results_dir, colormaps[1])
plot_1d_heatmap_anim(U1, T1, X1, "system_1_U", results_dir, colormaps[1])

In [ ]:
# Processing System 2
print("Processing system 2...")
U2, V2, X2, Y2, T2 = load_dataset(data_paths[2])
plot_1d_heatmap(U2, T2, X2, "system_2_U", results_dir, colormaps[2])
plot_1d_heatmap_anim(U2, T2, X2, "system_2_U", results_dir, colormaps[2])

In [ ]:
# Processing System 3
print("Processing system 3...")
U3, V3, X3, Y3, T3 = load_dataset(data_paths[3])
plot_2d_heatmap_anim(U3, T3, X3, Y3, "system_3_U", results_dir, colormaps[3])
if V3 is not None:
    plot_2d_heatmap_anim(V3, T3, X3, Y3, "system_3_V", results_dir, colormaps[3])

# Data Inspection

In [7]:
# Paths to data files
data_path_1 = "data/1/"
data_path_2 = "data/2/"
data_path_3 = "data/3/"

# Load data for File 1
t1 = np.load(data_path_1 + "t.npy")
x1 = np.load(data_path_1 + "x.npy")
u1 = np.load(data_path_1 + "u.npy")

# Load data for File 2
t2 = np.load(data_path_2 + "t.npy")
x2 = np.load(data_path_2 + "x.npy")
u2 = np.load(data_path_2 + "u.npy")

# Load data for File 3
t3 = np.load(data_path_3 + "t.npy")
x3 = np.load(data_path_3 + "x.npy")
y3 = np.load(data_path_3 + "y.npy")
u3 = np.load(data_path_3 + "u.npy")  # u(x, y, t)
v3 = np.load(data_path_3 + "v.npy")  # v(x, y, t)

In [9]:
# Inspect shapes of the data
print("File 1 Data Shapes: x:", x1.shape, "t:", t1.shape, "u:", u1.shape)
print("File 2 Data Shapes: x:", x2.shape, "t:", t2.shape, "u:", u2.shape)
print("File 3 Data Shapes: x:", x3.shape, "y:", y3.shape, "t:", t3.shape, "u:", u3.shape, "v:", v3.shape)

File 1 Data Shapes: x: (256, 101) t: (256, 101) u: (256, 101)
File 2 Data Shapes: x: (512, 201) t: (512, 201) u: (512, 201)
File 3 Data Shapes: x: (256, 256, 201) y: (256, 256, 201) t: (256, 256, 201) u: (256, 256, 201) v: (256, 256, 201)


# Discovery of System 1

In [12]:
# Create directory for results
results_dir = "PDE-1"
os.makedirs(results_dir, exist_ok=True)

In [14]:
# 1. Load Data
t1 = np.load("data/1/t.npy")
x1 = np.load("data/1/x.npy")
u1 = np.load("data/1/u.npy")

# 2. Normalize Data
u1_norm = u1 / np.max(np.abs(u1))

# 3. Compute Grid Spacing
dx1 = np.mean(np.diff(x1[:, 0]))  # Assuming uniform spacing in x
dt1 = np.mean(np.diff(t1[0, :]))  # Assuming uniform spacing in t

# 4. Build Linear System
D = 3  # Maximum derivative order
P = 3  # Maximum polynomial degree
ut1, Theta1, rhs_description1 = build_linear_system(
    u1_norm, 
    dt=dt1, 
    dx=dx1, 
    D=D, 
    P=P, 
    time_diff='FD', 
    space_diff='FD'
)

# 5. Sparse Regression
lambda_reg = 1e-5  # Regularization parameter
tolerance = 1e-6   # Sparsity tolerance
xi1 = TrainSTRidge(Theta1, ut1, lam=lambda_reg, d_tol=tolerance)

# 6. Output the Identified PDE
print("\nIdentified PDE for File 1:")
print_pde(xi1, rhs_description1, ut="u_t")

Optimal tolerance: 2.5000000000000008e-05

Identified PDE for File 1:


<IPython.core.display.Math object>

u_t =
    -1.01491   uu_{x}
     0.09949   u_{xx}


In [15]:
ut1.shape, Theta1.shape, len(rhs_description1), xi1.shape

((25856, 1), (25856, 16), 16, (16, 1))

In [16]:
# Size of D is directly reflected by the number of columns in the feature matrix Θ:
print("Size of Library D:", Theta1.shape[1])

Size of Library D: 16


In [17]:
# Step 1: Compute predicted u_t
u_t_pred = Theta1 @ xi1

# Step 2: Compute residual
residual = ut1 - u_t_pred

# Step 3: Compute MSE
mse = np.mean(residual**2)

# Step 4: Compute relative error
relative_error = np.linalg.norm(residual) / np.linalg.norm(ut1)

print("Mean Squared Error (MSE):", mse)
print("Relative Error:", relative_error)

Mean Squared Error (MSE): (3.3645527048940624e-07+0j)
Relative Error: 0.00878118607886049


In [18]:
ut1.shape, u_t_pred.shape

((25856, 1), (25856, 1))

In [23]:
ut1.shape, u_t_pred.shape

((25856, 1), (25856, 1))

In [26]:
u1.shape

(256, 101)

In [28]:
nx, nt = 256, 101

# Reshape the 1D arrays into 2D arrays using column-major order
ut1_reshaped = ut1.reshape((nx, nt), order='F')
u_t_pred_reshaped = u_t_pred.reshape((nx, nt), order='F')
abs_error = np.abs(ut1_reshaped - u_t_pred_reshaped)

# A spatial grid is extracted from the first column of x1 and a temporal grid from the first row of t1.
x_grid = x1[:, 0]
t_grid = t1[0, :]

In [30]:
heatmap_save_path = os.path.join(results_dir, "pde_1_ut_heatmap_comparison.png")
plot_comparison_heatmaps(ut1_reshaped, u_t_pred_reshaped, abs_error, x_grid, t_grid, heatmap_save_path)

Heatmap comparison saved as 'PDE-1/pde_1_ut_heatmap_comparison.png'


In [31]:
# Save the animation of the spatiotemporal comparison
pde_string = get_pde_string(xi1, rhs_description1, ut='u_t', precision=3, threshold=8e-2)
animate_save_path = os.path.join(results_dir, "pde_1_spatiotemporal_animation_comparison.gif")
animate_comparison(x_grid, t_grid, ut1_reshaped, u_t_pred_reshaped, pde_string, animate_save_path)

Animation saved as 'PDE-1/pde_1_spatiotemporal_animation_comparison.gif'


In [32]:
plot_pde_terms(rhs_description1, xi1, 'PDE-1/pde_1_bar_approximated_terms.png')

Term coefficients plot saved as 'PDE-1/pde_1_bar_approximated_terms.png'


In [33]:
n, m = u1.shape[0], u1.shape[1]
plot_candidate_terms(Theta1, rhs_description1, n, m, x_grid, t_grid, 'PDE-1/pde_1_candidate_terms.png')

Candidate terms plot saved as 'PDE-1/pde_1_candidate_terms.png'


In [34]:
scatter_true_vs_pred(ut1, u_t_pred, "PDE-1/pde_1_scatter_true_vs_predicted_ut.png", minor_tick_length=2)

Scatter plot over time plot saved as 'PDE-1/pde_1_scatter_true_vs_predicted_ut.png'


In [35]:
# Compute predicted u_t from the sparse regression model
u_t_pred_1 = Theta1 @ xi1

# Reshape both the original u_t and predicted u_t into a 2D grid.
# System 1 data have dimensions 256 x 101.
ut1_reshaped = ut1.reshape((256, 101), order='F')
u_t_pred_1_reshaped = u_t_pred_1.reshape((256, 101), order='F')

# Extract 1D spatial and temporal grids from x1 and t1.
# It is assumed that each column (or row) is constant.
x1_grid = x1[:, 0]
t1_grid = t1[0, :]

# Define a path for saving the residual plot.
residual_plot_path1 = os.path.join("PDE-1", "pde_1_residual_over_time.png")
os.makedirs(os.path.dirname(residual_plot_path1), exist_ok=True)

# Plot the residual over time using the previously defined function.
plot_residual_over_time(ut1_reshaped, u_t_pred_1_reshaped, t1_grid, residual_plot_path1)

Residual over time plot saved as 'PDE-1/pde_1_residual_over_time.png'


---

# Discovery of System 2

In [38]:
results_dir = "PDE-2"
os.makedirs(results_dir, exist_ok=True)

In [40]:
# Load data for File 2
t2 = np.load("data/2/t.npy")
x2 = np.load("data/2/x.npy")
u2 = np.load("data/2/u.npy")

print("File 2 Data Shapes:")
print(f"x: {x2.shape}, t: {t2.shape}, u: {u2.shape}")

File 2 Data Shapes:
x: (512, 201), t: (512, 201), u: (512, 201)


In [41]:
# Extract grid dimensions and compute grid spacing
n2, m2 = u2.shape
dt2 = np.mean(np.diff(t2, axis=1))  # Temporal grid spacing
dx2 = np.mean(np.diff(x2, axis=0))  # Spatial grid spacing

print(f"dx: {dx2}, dt: {dt2}")

dx: 0.1171875, dt: 0.10000000149011612


In [42]:
# Build the linear system
D = 3  # Maximum derivative order
P = 1  # Maximum polynomial order
ut2, Theta2, rhs_description2 = build_linear_system(u2, dt2, dx2, D=D, P=P, time_diff="FD", space_diff="FD")

print(f"Theta shape: {Theta2.shape}, ut shape: {ut2.shape}")
print("RHS Description:")
print(rhs_description2)

Theta shape: (102912, 8), ut shape: (102912, 1)
RHS Description:
['', 'u', 'u_{x}', 'uu_{x}', 'u_{xx}', 'uu_{xx}', 'u_{xxx}', 'uu_{xxx}']


In [43]:
# Train the sparse regression model
lambda_reg = 1e-5  # Regularization parameter
tolerance = 1e-4  # Sparsity tolerance
xi2 = TrainSTRidge(Theta2, ut2, lam=lambda_reg, d_tol=tolerance)

# Print the identified PDE
print("Identified PDE for File 2:")
print_pde(xi2, rhs_description2)

Optimal tolerance: 0
Identified PDE for File 2:


<IPython.core.display.Math object>

u_t =
    -5.56656   uu_{x}
    -0.88657   u_{xxx}
    -0.10044   uu_{xxx}


In [44]:
Theta2.shape

(102912, 8)

In [45]:
print("Size of Library D:", Theta2.shape[1])

Size of Library D: 8


In [46]:
# Compute predicted u_t
u_t_pred2 = Theta2 @ xi2

# Reshape for visualization
u_t_true_reshaped2 = np.real(ut2).reshape(n2, m2)
u_t_pred_reshaped2 = np.real(u_t_pred2).reshape(n2, m2)


# Mean Squared Error (MSE)
mse2 = np.mean((ut2 - u_t_pred2)**2)

# Mean Absolute Error (MAE)
mae2 = np.mean(np.abs(ut2 - u_t_pred2))

# Relative Average L2 Error
relative_l2_error2 = np.linalg.norm(ut2 - u_t_pred2) / np.linalg.norm(ut2)

# Print the errors
print("Mean Squared Error (MSE) for File 2:", mse2)
print("Mean Absolute Error (MAE) for File 2:", mae2)
print("Relative Average L2 Error for File 2:", relative_l2_error2)

Mean Squared Error (MSE) for File 2: (6.8853947e-06+0j)
Mean Absolute Error (MAE) for File 2: 0.0017760391
Relative Average L2 Error for File 2: 0.054540124


In [47]:
plot_pde_terms(rhs_description2, xi2, 'PDE-2/pde_2_bar_approximated_terms.png')

Term coefficients plot saved as 'PDE-2/pde_2_bar_approximated_terms.png'


In [48]:
# Compute the predicted u_t for system 2
u_t_pred_2 = Theta2 @ xi2

# Reshape u_t arrays to 2D grids (512 x 201) using Fortran order
ut2_reshaped = ut2.reshape((512, 201), order='F')
u_t_pred_2_reshaped = u_t_pred_2.reshape((512, 201), order='F')

# For the spatial and temporal axes, extract 1D grids.
# We assume that each column (or row) in x2 and t2 is constant.
x2_grid = x2[:, 0]
t2_grid = t2[0, :]

In [49]:
results_dir = "PDE-2"

# --- 1. Heatmap Comparison ---
os.makedirs(results_dir, exist_ok=True)
heatmap_save_path = os.path.join(results_dir, "pde_2_ut_heatmap_comparison.png")

# Plot the comparison heatmaps for u_t:
plot_comparison_heatmaps(ut2_reshaped, u_t_pred_2_reshaped, 
                         np.abs(ut2_reshaped - u_t_pred_2_reshaped),
                         x2_grid, t2_grid, heatmap_save_path)

# --- 2. Animation Comparison ---
pde_string_2 = get_pde_string(xi2, rhs_description2, ut='u_t', precision=3, threshold=8e-2)
animate_save_path = os.path.join(results_dir, "pde_2_spatiotemporal_animation_comparison.gif")
animate_comparison(x2_grid, t2_grid, ut2_reshaped, u_t_pred_2_reshaped, pde_string_2, animate_save_path)

# --- 3. PDE Term Coefficients ---
os.makedirs(results_dir, exist_ok=True)
terms_save_path = os.path.join(results_dir, "pde_2_bar_approximated_terms.png")
plot_pde_terms(rhs_description2, xi2, terms_save_path)

# --- 4. Candidate Terms Heatmaps ---
os.makedirs(results_dir, exist_ok=True)
candidate_save_path = os.path.join(results_dir, "pde_2_candidate_terms.png")
plot_candidate_terms(Theta2, rhs_description2, n2, m2, x2_grid, t2_grid, candidate_save_path)


Heatmap comparison saved as 'PDE-2/pde_2_ut_heatmap_comparison.png'
Animation saved as 'PDE-2/pde_2_spatiotemporal_animation_comparison.gif'
Term coefficients plot saved as 'PDE-2/pde_2_bar_approximated_terms.png'
Candidate terms plot saved as 'PDE-2/pde_2_candidate_terms.png'


In [50]:
scatter_true_vs_pred(ut2, u_t_pred_2, "PDE-2/pde_2_scatter_true_vs_predicted_ut.png", minor_tick_length=4)

Scatter plot over time plot saved as 'PDE-2/pde_2_scatter_true_vs_predicted_ut.png'


In [51]:
residual_plot_path = os.path.join("PDE-2", "pde_2_residual_over_time.png")
plot_residual_over_time(ut2_reshaped, u_t_pred_2_reshaped, t2_grid, residual_plot_path)

Residual over time plot saved as 'PDE-2/pde_2_residual_over_time.png'


---

# Discovery of System 3

## Original:

In [18]:
# Load data for File 3
t3 = np.load(data_path_3 + "t.npy")
x3 = np.load(data_path_3 + "x.npy")
y3 = np.load(data_path_3 + "y.npy")
u3 = np.load(data_path_3 + "u.npy")  # u(x, y, t)
v3 = np.load(data_path_3 + "v.npy")  # v(x, y, t)

# Print original shapes
print("Original File 3 Data Shapes:")
print(f"x: {x3.shape}, y: {y3.shape}, t: {t3.shape}, u: {u3.shape}, v: {v3.shape}")

# Compute grid spacings from the original arrays (assuming the grid is uniform and constant along other dimensions)
dx3 = np.mean(np.diff(x3[:, 0, 0]))
dy3 = np.mean(np.diff(y3[0, :, 0]))
dt3 = np.mean(np.diff(t3[0, 0, :]))
print(f"Original dx: {dx3}, dy: {dy3}, dt: {dt3}")

# Build the linear system for the full (non-downsampled) data
U_t3, Theta3, rhs_description3 = build_linear_system_2D(u3, v3, dt3, dx3, dy3, D=3, P=3, time_diff="FD", space_diff="FD")
print(f"Original U_t3 shape: {U_t3.shape}, Theta3 shape: {Theta3.shape}")
print("RHS Description:", rhs_description3)


Original File 3 Data Shapes:
x: (256, 256, 201), y: (256, 256, 201), t: (256, 256, 201), u: (256, 256, 201), v: (256, 256, 201)
Original dx: 0.078125, dy: 0.078125, dt: 0.05000000074505806
Original U_t3 shape: (13172736, 2), Theta3 shape: (13172736, 10)
RHS Description: ['', 'u', 'v', 'u^2', 'v^2', 'uv', 'u_x', 'u_y', 'v_x', 'v_y']


## Downsampling:

In [20]:
# Set the downsampling factor (e.g., 4)
ds_factor = 4

# Downsample u3, v3, x3, y3, t3
u3_ds = downsample_3d(u3, ds_factor)
v3_ds = downsample_3d(v3, ds_factor)
x3_ds = downsample_3d(x3, ds_factor)
y3_ds = downsample_3d(y3, ds_factor)
t3_ds = downsample_3d(t3, ds_factor)

print("Downsampled Data Shapes:")
print(f"x: {x3_ds.shape}, y: {y3_ds.shape}, t: {t3_ds.shape}, u: {u3_ds.shape}, v: {v3_ds.shape}")

# Recompute grid spacings on the downsampled grid:
dx3_ds = np.mean(np.diff(x3_ds[:,0,0]))
dy3_ds = np.mean(np.diff(y3_ds[0,:,0]))
dt3_ds = np.mean(np.diff(t3_ds[0,0,:]))
print(f"Downsampled dx: {dx3_ds}, dy: {dy3_ds}, dt: {dt3_ds}")

# Optionally, normalize the downsampled fields
u3_ds_norm = u3_ds / np.max(np.abs(u3_ds))
v3_ds_norm = v3_ds / np.max(np.abs(v3_ds))

# Build the linear system on the downsampled data
U_t3_ds, Theta3_ds, rhs_description3_ds = build_linear_system_2D(u3_ds_norm, v3_ds_norm, 
                                                                 dt3_ds, dx3_ds, dy3_ds, 
                                                                 D=3, P=3, time_diff="FD", space_diff="FD")
print(f"Downsampled U_t3 shape: {U_t3_ds.shape}, Theta3 shape: {Theta3_ds.shape}")
print("RHS Description:", rhs_description3_ds)

# Now, training the sparse regression on Theta3_ds and U_t3_ds will be much faster.
lambda_reg = 1e-5
tolerance = 1e-5
Xi3_ds = TrainSTRidge_multioutput(Theta3_ds, U_t3_ds, lam=lambda_reg, d_tol=tolerance)
print("Xi3 shape (downsampled):", Xi3_ds.shape)

Downsampled Data Shapes:
x: (64, 64, 51), y: (64, 64, 51), t: (64, 64, 51), u: (64, 64, 51), v: (64, 64, 51)
Downsampled dx: 0.3125, dy: 0.3125, dt: 0.20000000298023224
Downsampled U_t3 shape: (208896, 2), Theta3 shape: (208896, 10)
RHS Description: ['', 'u', 'v', 'u^2', 'v^2', 'uv', 'u_x', 'u_y', 'v_x', 'v_y']
Xi3 shape (downsampled): (10, 2)


In [22]:
# 1. Print the identified PDEs for the two equations using the downsampled data
print("Identified PDE for File 3 (u_t equation, downsampled):")
print_pde(Xi3_ds[:, 0:1], rhs_description3_ds, ut="u_t")
print("Identified PDE for File 3 (v_t equation, downsampled):")
print_pde(Xi3_ds[:, 1:2], rhs_description3_ds, ut="v_t")

# 2. Compute predictions on the downsampled candidate matrix
U_t3_ds_pred = Theta3_ds @ Xi3_ds  # shape: (N_ds, 2), where N_ds is the total number of downsampled samples

# 3. Reshape predictions back into the downsampled grid
# For File 3, original shape was (256,256,201); with downsampling factor 4, new shape is (64, 64, nt_ds)
nx_ds, ny_ds, nt_ds = u3_ds.shape  # e.g., (64, 64, 50) if 201/4 ≈ 50
u_t3_ds_pred = U_t3_ds_pred[:, 0].reshape((nx_ds, ny_ds, nt_ds), order='F')
v_t3_ds_pred = U_t3_ds_pred[:, 1].reshape((nx_ds, ny_ds, nt_ds), order='F')

# 4. Compute the true time derivatives on the downsampled data (using the same finite difference method)
u_t3_ds_true = np.gradient(u3_ds_norm, dt3_ds, axis=2)
v_t3_ds_true = np.gradient(v3_ds_norm, dt3_ds, axis=2)

# 5. Compute error metrics:
# L2 error
err_u = np.linalg.norm(u_t3_ds_true - u_t3_ds_pred) / np.linalg.norm(u_t3_ds_true)
err_v = np.linalg.norm(v_t3_ds_true - v_t3_ds_pred) / np.linalg.norm(v_t3_ds_true)
# Compute Mean Squared Error (MSE)
mse_u = np.mean((u_t3_ds_true - u_t3_ds_pred)**2)
mse_v = np.mean((v_t3_ds_true - v_t3_ds_pred)**2)
print()
print(f"Relative L2 Error for u_t (downsampled): {err_u:.2e}")
print(f"Relative L2 Error for v_t (downsampled): {err_v:.2e}")
print(f"MSE for u_t (downsampled): {mse_u:.2e}")
print(f"MSE for v_t (downsampled): {mse_v:.2e}")

Identified PDE for File 3 (u_t equation, downsampled):


<IPython.core.display.Math object>

u_t =
     0.88532   v
Identified PDE for File 3 (v_t equation, downsampled):


<IPython.core.display.Math object>

v_t =
    -0.88544   u

Relative L2 Error for u_t (downsampled): 1.50e-01
Relative L2 Error for v_t (downsampled): 1.51e-01
MSE for u_t (downsampled): 8.06e-03
MSE for v_t (downsampled): 8.11e-03


In [ ]:
results_dir = "PDE-3"

# Extract 1D grids from the downsampled arrays.
# We assume the grid values are constant along the other dimensions.
x3_ds_grid = x3_ds[:, 0, 0]   # 1D spatial grid for x, shape (nx_ds,)
y3_ds_grid = y3_ds[0, :, 0]   # 1D spatial grid for y, shape (ny_ds,)
t3_ds_grid = t3_ds[0, 0, :]   # 1D time grid, shape (nt_ds,)

# Animation for u_t using the downsampled data:
animate_comparison_heatmaps_2D(u_t3_ds_true, u_t3_ds_pred, 
                               x3_ds_grid, y3_ds_grid, t3_ds_grid, 
                               "u_t", results_dir, "jet")

In [ ]:
# Animation for v_t using the downsampled data:
animate_comparison_heatmaps_2D(v_t3_ds_true, v_t3_ds_pred, 
                               x3_ds_grid, y3_ds_grid, t3_ds_grid, 
                               "v_t", results_dir, "jet")

In [25]:
os.makedirs(results_dir, exist_ok=True)

plot_pde_terms_multioutput(rhs_description3_ds, Xi3_ds,
                           os.path.join(results_dir, "pde_3_bar_terms_ut.png"),
                           os.path.join(results_dir, "pde_3_bar_terms_vt.png"))

Term coefficients for u_t plot saved as 'PDE-3/pde_3_bar_terms_ut.png'
Term coefficients for v_t plot saved as 'PDE-3/pde_3_bar_terms_vt.png'


In [27]:
# Extract 1D grids for x and t from downsampled x3_ds and t3_ds:
x3_ds_grid = x3_ds[:, 0, 0]    # shape (nx_ds,)
t3_ds_grid = t3_ds[0, 0, :]    # shape (nt_ds,)

# Use the middle index for y:
y_index = ny_ds // 2

# Save candidate terms plot for File 3
save_path = os.path.join("PDE-3", "pde_3_candidate_terms_downsampled.png")
os.makedirs("PDE-3", exist_ok=True)
plot_candidate_terms_PDE3(Theta3_ds, rhs_description3_ds, nx_ds, ny_ds, nt_ds, 
                          x3_ds_grid, t3_ds_grid, y_index, save_path)

Candidate terms plot saved as 'PDE-3/pde_3_candidate_terms_downsampled.png'


In [29]:
nx, ny, nt = u3.shape  # For File 3: (256, 256, 201)
x3_grid = x3[:, 0, 0]   # 1D grid for x, length 256
t3_grid = t3[0, 0, :]   # 1D grid for t, length 201

# Choose a y-index to fix (for example, the middle index)
y_index = ny // 2  # 128 if ny is 256

# Set a save path for the candidate terms figure for the full data.
save_path_full = os.path.join("PDE-3", "pde_3_candidate_terms.png")
os.makedirs("PDE-3", exist_ok=True)

# Plot candidate terms using the full (non-downsampled) candidate matrix Theta3.
plot_candidate_terms_PDE3(Theta3, rhs_description3, nx, ny, nt, x3_grid, t3_grid, y_index, save_path_full)

Candidate terms plot saved as 'PDE-3/pde_3_candidate_terms.png'


In [31]:
# For the full data:
nx, ny, nt = u3.shape  # e.g. (256, 256, 201)
x3_grid = x3[:, 0, 0]
y3_grid = y3[0, :, 0]
t_index = nt // 2

save_path = os.path.join("PDE-3", "pde_3_candidate_terms_spatial_full.png")
os.makedirs("PDE-3", exist_ok=True)
plot_candidate_terms_spatial(Theta3, rhs_description3, nx, ny, nt, x3_grid, y3_grid, t_index, save_path)

Spatial candidate terms plot saved as 'PDE-3/pde_3_candidate_terms_spatial_full.png'


In [33]:
# Plot residual over time for u_t (downsampled)
residual_plot_path_ut = os.path.join(results_dir, "pde_3_residual_over_time_ut.png")
plot_residual_over_time_2D(u_t3_ds_true, u_t3_ds_pred, t3_ds_grid, residual_plot_path_ut, variable_label="u_t")

# Plot residual over time for v_t (downsampled)
residual_plot_path_vt = os.path.join(results_dir, "pde_3_residual_over_time_vt.png")
plot_residual_over_time_2D(v_t3_ds_true, v_t3_ds_pred, t3_ds_grid, residual_plot_path_vt, variable_label="v_t")


Residual over time plot for u_t saved as 'PDE-3/pde_3_residual_over_time_ut.png'
Residual over time plot for v_t saved as 'PDE-3/pde_3_residual_over_time_vt.png'


In [ ]:
# Call the function for the PDE-3 u_t plot
scatter_true_vs_pred_with_footnote(
    true=u_t3_ds_true,
    pred=u_t3_ds_pred,
    xlabel="True $u_t$",
    ylabel="Reconstructed $u_t$",
    title="True vs Reconstructed $u_t$ | PDE-3",
    file_name="PDE-3/pde_3_scatter_true_vs_predicted_ut.png",
    minor_tick_length=4,
    footnote="* All spatial and temporal points pooled"
)

# Call the function for the PDE-3 v_t plot
scatter_true_vs_pred_with_footnote(
    true=v_t3_ds_true,
    pred=v_t3_ds_pred,
    xlabel="True $v_t$",
    ylabel="Reconstructed $v_t$",
    title="True vs Reconstructed $v_t$ | PDE-3",
    file_name="PDE-3/pde_3_scatter_true_vs_predicted_vt.png",
    minor_tick_length=4,
    footnote="* All spatial and temporal points pooled"
)

In [37]:
results_dir = "PDE-3"
os.makedirs(results_dir, exist_ok=True)

# Extract 1D grid for x (from downsampled data)
x3_ds_grid = x3_ds[:, 0, 0]  # length = nx_ds
t3_ds_grid = t3_ds[0, 0, :]  # length = nt_ds

# Choose a fixed y slice, e.g., the middle index
y_index = y3_ds.shape[1] // 2

save_path_ut = os.path.join(results_dir, "pde_3_animate_ut_slice.gif")
animate_comparison_2D_slice(x3_ds_grid, t3_ds_grid, u_t3_ds_true, u_t3_ds_pred,
                            y_index, save_path_ut, variable_label="u_t", duration=0.1)

save_path_vt = os.path.join(results_dir, "pde_3_animate_vt_slice.gif")
animate_comparison_2D_slice(x3_ds_grid, t3_ds_grid, v_t3_ds_true, v_t3_ds_pred,
                            y_index, save_path_vt, variable_label="v_t", duration=0.1)

Animation saved as 'PDE-3/pde_3_animate_ut_slice.gif'
Animation saved as 'PDE-3/pde_3_animate_vt_slice.gif'
